## 설정

In [1]:
import shutil
import datetime
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

from chart import *
from table import *
from docx.text.run import *

In [2]:
# DRM 방지(default.docx)
def findfile(name, path):
    for dirpath, dirname, filename in os.walk(path):
        if name in filename:
            return os.path.join(dirpath, name)

file_path=os.path.abspath("")
srcpath = findfile("default.docx.org", file_path)
dir, file = os.path.split(srcpath)
shutil.copy2(srcpath, dir+"\default.docx")

'C:\\work\\rpa\\venv\\Lib\\site-packages\\docx\\templates\\default.docx'

In [3]:
from document import *
# report version
rtype = '월간' # 연간, 반기

In [36]:
region_list = [ 'KR', 'CN', 'NA', 'EU', 'SG', 'RU', 'IN' ]
# region = 'All'
# region = 'KR' # All|KR|CN|NA|EU|SG|RU
# region = 'CN'
# region = 'NA'
region = 'EU'
# region = 'SG'
# region = 'RU'
# RU쪽은 SG로 나오는 버그 존재

In [37]:
month = '8월'

In [38]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

v = '1'

if region == 'All':
    regex = '(KR|CN|NA|EU|SG|RU|IN|Global|All)'
else:
    regex = region

In [39]:
# Document 기본 폰트
style = document.styles['Normal']
# style.font.name = '맑은고딕'
style.font.name = 'Calibri'
style.font.size = Pt(12)
# style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은고딕')
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Calibri')

In [40]:
# Chart 기본 폰트
alt.themes.register('맑은고딕', hanfont)
alt.themes.enable('맑은고딕')

ThemeRegistry.enable('맑은고딕')

## 주요 이벤트

In [9]:
# ETL
base_path = r"./data/요약_서비스관리_v1.ods"
#base_path = r"./data/주요이벤트_202206_서비스관리_0811_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed_rows = len(df_completed)
completed = counter(df_completed, regex)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted_rows = len(df_incompleted)
incompleted = counter(df_incompleted, regex)

In [10]:
print(completed, incompleted)

6 14


In [11]:
# df_filtered.loc[(df_filtered['월']==month)]
# df_filtered[(df_filtered['월']==month)]

In [12]:
# # Chart
# if region == 'All':
#     # Data
#     rows = df_filtered.loc[(df_filtered['월']==month)]
    
#     data, pivot, total = getPivotTable_event(rows, month, region_list)
#     source = pivot
    
#     # Chart
#     if incompleted == 0:
#         chart = getPieChart(source)
#     else:
#         source = flatten_2d(data)
#         chart = getStackedHBarChart(source)
#     chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
#     document.add_paragraph('주요 이벤트')
#     # Document
#     document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
#     document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
# else:
#     document.add_paragraph('주요 이벤트')
#     document.add_paragraph(f'완료: {completed} 건 / 이슈: {incompleted} 건', style='List Bullet')
#     pass

In [13]:
# # source
# # df.groupby('리전')['월'].agg(**{'합계_test':'sum'}).reset_index()
# df.groupby('리전')['월'].count().reset_index()
# df.groupby('Work')['

In [14]:
# Chart
if region == 'All':
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
    # document.add_paragraph('주요 이벤트')
    document.add_heading('주요 이벤트', level=1)
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    document.add_heading('주요 이벤트', level=1)
    t = completed + incompleted
    if t == completed :
        if t == 0 : 
            document.add_paragraph(f'총 {t} 건', style='List Bullet')
        else:
            document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
    else :
        document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')


    pass

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [15]:
# def getStackedHBarChart(source):
#     bar = alt.Chart(source).mark_bar().encode(
#         x=alt.X('count(리전):Q', stack='zero', title=''),         
#         y=alt.Y('진행상태:O', sort='ascending'),  
#         # order=alt.Order("count(리전):Q", sort='descending'),
#         color=alt.Color('리전:N') #, legend=None)   
#     )
    
#     text1 = alt.Chart(source).mark_text(dx=-20, dy=0, color='black').encode(
#         x=alt.X('count(리전):Q', stack='zero', sort='ascending'),        
#         y=alt.Y('진행상태:N', sort='ascending'),        
#         detail=alt.Color("리전:N"),
#         text = alt.Text('count(리전):Q', format='d') #, sort='descending')
#     )
    
#     text2 = alt.Chart(source).mark_text(dx=10, dy=-3, color='black').encode(
#         x=alt.X('count(리전):Q', stack='zero', title=''),         
#         y=alt.Y('진행상태:N'),        
#         # order=alt.Order("count(리전):Q", sort='descending'),
#         # color=alt.Color('리전:N'), # legend=None)   
#         text = alt.Text('count(리전):Q', format='d')
#         # text = alt.Text('sum(count):Q', format='d')        
#     ) 
    
#     chart = (bar + text1 + text2).properties(width=360, height=360)
#     return chart

In [16]:
# df_filtered

In [17]:
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addSummaryTable(df_filtered, regex, month)
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [18]:
document.save('demo.docx')

## 1.1 모니터링(인시던트핸들링)

In [19]:
base_path = r"./data/1_1_모니터링_인시던트핸들링_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df['장애전파소요시간'] = (pd.to_datetime(df['장애전파시간.1'])-pd.to_datetime(df['발생인지시간'])).astype('timedelta64[m]')

# Pivoting
pivot, total = getPivotTable(df, month)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered, timeslot = getTables(df_filtered, regex, month)
    
# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 15) & (df_filtered['월']== month)]
df_failed[['날짜', '장애이벤트', '조치내용', 'JIRA_Ticket_No', '장애전파소요시간']]

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [20]:
print(completed, incompleted)

266 0


In [21]:
# Chart - Pie 차트
if region == 'All':
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('모니터링(인시던트핸들링)', level=1) 
    #document.add_paragraph('1. 모니터링(인시던트핸들링)')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    # chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('모니터링(인시던트핸들링)', level=1) 
    t = completed + incompleted
    # document.add_paragraph(f'완료: {completed} 건 / 이슈: {incompleted} 건', style='List Bullet')
    document.add_paragraph(f'총 {t} 건 ', style='List Bullet')
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')

In [22]:
# Document
# document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
# if region == 'All':
    # document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
# else:
    # pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

# Document
document.add_paragraph(f'일별 장애전파 소요시간(분) 현황', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

##  (향후) 평균 소요 시간, 최대 소요 시간 추가 하자!!!
document.add_paragraph(f'장애전파SLA(15분) 초과: {len(df_failed)} 건', style='List Bullet')
if df_failed.empty :
    pass
else:
    addFailedTable(df_failed, regex, month)

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)
document.add_page_break()


## 1.2 이슈 관리

In [23]:
base_path = r"./data/1_2_이슈관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & ((df['진행상태']=='완료')|(df['진행상태']=='진행중')) & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [24]:
print(completed, incompleted)

2 4


In [25]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)
        
    chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('이슈관리', level=1) 
    #document.add_paragraph('2. 이슈관리')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    # chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('이슈관리', level=1) 
    t = completed + incompleted
    if t == completed :
        if t == 0 : 
            document.add_paragraph(f'총 {t} 건', style='List Bullet')
        else:
            document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
    else :
        document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')

    # document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    pass


# Chart - 라인 차트
data = counter_3d(df_filtered, month)
chart = getLineChart(data)
# chart = getLineChart2(df_filtered, month)
# chart.show()
chart.save(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

In [26]:
# Document
total_sum = completed + incompleted
# document.add_paragraph('이슈 관리', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')

# if region == 'All':
#     document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# Document
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable4(df_filtered, regex, month)
# addTable4(df_filtered, regex, month)

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

## 1.3 장애 관리

In [27]:
base_path = r"./data/1_3_장애관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [28]:
print(completed, incompleted)

4 0


In [29]:
df_filtered

,년도,월,리전,AZ,테넌트,진행상태,성패,제목,System_Service,Date,Engineer,Root_Cause,Impact_Severity,작업내용,Ticket,ReviewDated,ReviewResult,주간,월간,연간
0,2022년,1월,KR,1센터,PRD,완료,성공,해당 호스트 VM LB 인입 차단 후 호스트 리부팅,krcloudprdcomdr1510,2022-01-19T00:00:00,Subramanya Ambale,None,순차별 LB 절체로 인한 영향도 없음,self heal 프로세스 진행을 위한 호스트 리붓 필요\n1/19 GIS LB 긴...,KRCLOUD-4117,2022-01-19T00:00:00,self heal 프로세스(호스트 리부팅) 완료,O,O,O
2,2022년,2월,KR,1센터,PRD,완료,성공,KRCloud-PRD-IPMI-R14 Ethernet15 구간 확인 요청\nkrcl...,krcloudprdcomdR1413,2022-02-04T00:00:00,심명보\n우리 올제 김성균 대리,None,Motherboard 교체,21/12/04 KRCloud-PRD-IPMI-R14(10.11.41.194 ) -...,KRCLOUD-3838\nKRCLOUD-4135,None,None,O,O,O
3,2022년,2월,KR,1센터,PRD,완료,성공,Netapp 볼륨 점검 및 벤더 케이스분석 의뢰,netapp(10.11.41.230),2022-01-12T00:00:00,정종인\n이테크시스템 허정수 과장,None,서비스허브 connect fail증가,22/01/12 서비스허브 트래픽 밀림 현상 발생\n==> L4 및 트래픽 점검\n...,KRCLOUD-4128\nKRCLOUD-4122 \nKRCLOUD-4124,None,None,O,O,O
4,2022년,3월,KR,1센터,PRD,완료,성공,ccskrbigps03dnp02 리부팅 관련\nBAREMETAL krcloudprd...,ccskrbigps03dnp02 (10.11.63.239)\nkrcloudprdco...,2022-02-13T00:00:00,심명보\n우리올제 김성규 과장,None,krcloudprdcomdr1803 메모리 에러 로그 발생,22/02/13 BAREMETAL ccskrbigps03dnp02 vm 리부팅 발생...,KRCLOUD-4272,None,None,O,O,O
5,2022년,3월,KR,1센터,PRD,완료,성공,"통합계정 VM(account, profile) 장애관련 긴급 지원",UW-KRMTS-L7,2022-02-25T00:00:00,정종인 매니저,None,None,통합계정 장애 관련 변경 및 지원\n변경사항(22/02/25금)\n\nVM Scal...,KRCLOUD-4390\nKRCLOUD-4391\nKRCLOUD-4392,None,None,O,O,O
6,2022년,3월,NA,1센터,PRD,완료,성공,북미 데이터 센터 쿨러 이상 발생,NA Datacenter,2022-03-11T00:00:00,"정진호, 심명보, 이한새, 채한길, 우전성, Ahmed. 김의현\n/ 벤더사 직원",None,None,NACLOUD 일부 호스트 온도 알람 발생확인\nipmi 확인 시 온도 상승 확인\...,NACLOUD-1263,None,None,O,O,O
8,2022년,4월,CN,1센터,PRD,완료,성공,NBU장비 출구 라우팅을 변경.\n접속 않되는거에 대하여 수동으로 라우팅을 추가.,ccscntmsdbp01 (10.124.253.152),2022-04-05T00:00:00,중국 법인 최영근\nCCS 이한새,None,netbackup 수행 안 됨.,22/04/06 넷백업 수행 안 되는 현상 발생\n22/04/08 넷백업 스위치 d...,CNCLOUD-1931\nCNCLOUD-1948,None,None,O,O,O
9,2022년,4월,NA,1센터,PRD,완료,성공,DB Recovery,ccsnakmatmsdbp01\nccsnakmatmsdbp02,2022.04.18 07:44 KST,김효은 매니저\n이한새 매니저\n정진호 매니저\nT-max 이정훈 수석\nVCS 이...,DB block corruption,DB Down,DB block이 깨지면서 해당 block read시 DB down현상 발생\nCC...,NACLOUD-1336,None,None,O,O,O
10,2022년,5월,KR,2센터,PRD,완료,성공,Host reboot,kr-az2-com06-01r08.kr.hcloud.h53,2022-05-12T00:00:00,None,Host reboot 발생으로 VM 13대 down time 발생,VM down time 발생\n\n(이중화 되어 있는 redis DB가 해당 호스트...,Evacuation 호스트 VM 이관 완료 후 VM 정상상태 확인\n\n영향받은 V...,None,None,None,O,O,O
11,2022년,5월,KR,2센터,PRD,완료,성공,Rancher 서비스 down,paas-prd-rancher02,2022-05-12T00:00:00,None,paas-prd-rancher02 해당 rancher VM의 메모리사용률이 90%를...,rancher 서비스 down,paas-prd-rancher 서비스 정상화 및 paas-prd-rancher02 ...,None,None,None,O,O,O


In [30]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('장애(RCA)관리', level=1) 
    #document.add_paragraph('3. 장애(RCA)관리')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    # chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('장애(RCA)관리', level=1) 
    t = completed + incompleted
    if t == completed :
        if t == 0 : 
            document.add_paragraph(f'총 {t} 건', style='List Bullet')
        else:
            document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
    else :
        document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')


    # document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    pass


# Chart - 라인 차트
data = counter_3d(df_filtered, month)
# data = { '월' : ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월'], '합계' : [0, 0, 0, 0, 0, 0, 0, 0] }
# data = pd.DataFrame(data)
chart = getLineChart(data)
chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')

In [31]:
df_filtered[df_filtered['월']==month]

,년도,월,리전,AZ,테넌트,진행상태,성패,제목,System_Service,Date,Engineer,Root_Cause,Impact_Severity,작업내용,Ticket,ReviewDated,ReviewResult,주간,월간,연간
21,2022년,8월,KR,1센터,PRD,완료,성공,krtmsprdcomh20\n\nhost down,ccskroapiapigwp03 - (192.168.46.254)\n\nccskre...,22/08/03 \n,이후승 매니저\n\n장재봉 매니저,None,krtmsprdcomh20 호스트 다운 알람 발생\n\n호스트 및 VM 재기동 후 ...,22/08/03 11:10 krtmsprdcomh20 호스트 다운\n22/08/03...,None,None,22/05/03 tibero 엔지니어와 복구 작업 후 log 분석 요청\n22/05...,O,O,O
22,2022년,8월,KR,1센터,PRD,완료,성공,RND 호스트\n\nkrrndprdcom11 다운 발생,krrndprdcom11\n(10.12.52.31),2022-08-11T00:00:00,심명보 매니저,None,krrndprdcom11 다운 알람 발생\n\n호스트 재기동 조치 실패\n\n일부 ...,22.08.11 15:00 krrndprdcom11 다운 발생\nkrrndprdc...,KRCLOUDBIG-859,None,22/05/03 tibero 엔지니어와 복구 작업 후 log 분석 요청\n22/05...,O,O,O
23,2022년,8월,KR,1센터,PRD,완료,성공,vpx-prd-a RNAT 세션 증가 및 1센터 DB full connection으...,vpx-prd-a (172.16.8.111),2022-08-09T00:00:00,정종인 매니저,None,1센터 DB connection full 발생하여 서비스담당자로부터 확인요청 \n\...,22/08/09 23:30 1센터 db connection 세션이 full차서 장...,None,None,22/05/03 tibero 엔지니어와 복구 작업 후 log 분석 요청\n22/05...,O,O,O
24,2022년,8월,KR,1센터,PRD,완료,성공,호스트 다운 발생\nkrcloudprdcomdr1615,krcloudprdcomdr1615,22/08/15 \n,고희경 매니저\n\n이시형 매니저\n\n채한길 매니저\n\n심명보 매니저,None,krcloudprdcomdr1615 다운 알람 발생\n\nipmi 확인시 메모리 에...,22.08.15 12:03 krcloudprdcomdr1615 다운 발생\nkr...,KRCLOUD-5459,None,22/05/03 tibero 엔지니어와 복구 작업 후 log 분석 요청\n22/05...,O,O,O


In [32]:
# Document
#document.add_paragraph('장애(RCA) 관리', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')

# if region == 'All':
#     document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')



p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# Document
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable4(df_filtered, regex, month)
# addTable4(df_filtered, regex, month)

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

In [33]:
document.save('demo.docx')

## 1.4 변경 관리

In [34]:
base_path = r"./data/1_4_변경관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, str(x)) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [35]:
print(completed, incompleted)

40 0


In [36]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('변경관리', level=1) 
    #document.add_paragraph('4. 변경관리')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    # chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('변경관리', level=1) 
    t = completed + incompleted
    if t == completed :
        if t == 0 : 
            document.add_paragraph(f'총 {t} 건', style='List Bullet')
        else:
            document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
    else :
        document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')

    # document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    pass


# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [37]:
df_filtered = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
total = len(df_filtered)

In [38]:
# Document
# document.add_paragraph('변경 관리', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 진행: {incompleted})', style='List Bullet')
    
# if region == 'All':
#     document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass

document.add_paragraph('월별 추세', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

# document.add_paragraph('상세 현황', style='List Bullet')

# if df_filtered[df_filtered['월']==month].empty :
#     pass
# else:
#     addTable3(df_filtered, regex, month)
# addTable3(df_filtered, regex, month)

# Document
p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# document.add_paragraph('월별 추세', style='List Bullet')
# document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph('상세 현황', style='List Bullet')

if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable3(df_filtered, regex, month)


document.add_page_break()

In [39]:
document.save('demo.docx')

## 1.5 요청 관리

In [40]:
base_path = r"./data/1_5_요청관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered2 = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [41]:
print(completed, incompleted)

745 0


In [42]:
df_filtered2

,년,월,리전,AZ,테넌트,진행상태,성패,제목,Ticket,Work Type,주간,월간,연간
0,22년,1월,CN,1센터,PRD,완료,성공,중국 PRD_CCS DB ZONE VM 배치,CNCLOUD-1303,None,O,O,O
5,22년,1월,CN,1센터,PRD,완료,성공,The application is at 10.109 135.15 install do...,CNCLOUD-1698,None,O,O,O
6,22년,1월,CN,1센터,PRD,완료,성공,BLE applets project IRdeto program using GW se...,CNCLOUD-1699,None,O,O,O
7,22년,1월,CN,1센터,PRD,완료,성공,Public IP Address IP address for accessing the...,CNCLOUD-1700,None,O,O,O
9,22년,1월,CN,1센터,PRD,완료,성공,검증계 및 운영계 Application Catalog 서비스 접속을 위해 방화벽 적...,CNCLOUD-1706,None,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,22년,8월,SG,1센터,PRD,완료,성공,[SG] LB 생성 요청,SGCLOUD-1213,Network,O,O,O
5028,22년,8월,SG,1센터,PRD,완료,성공,[SG] 운영계 LB 생성 요청,SGCLOUD-1214,Network,O,O,O
5029,22년,8월,SG,1센터,PRD,완료,성공,유럽 CCS 방화벽 해제 요청,SGCLOUD-1217,Security,O,O,O
5030,22년,8월,SG,1센터,PRD,완료,성공,싱가포르 CCS방화벽 해제 요청,SGCLOUD-1218,Security,O,O,O


In [43]:
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O') & (df_filtered['월']==month)]
df_filtered['Work Type'] = df_filtered['Work Type'].astype(str)
# df_filtered

In [44]:
# source
# df.groupby('리전')['월'].agg(**{'합계_test':'sum'}).reset_index()
# df.groupby('리전')['월'].count().reset_index()
# a=df.groupby('Work Type')['월'].count().reset_index().columns['Work Type','합계']
# df_filtered['Work Type'] = df_filtered['Work Type'].astype(str)
# a=df_filtered.groupby('Work Type')['월'].count().reset_index()
# a.rename(columns = {'월':'합계'},inplace=True)
# # print(a.to_frame(name='A'))
# a



# data = pd.DataFrame(a)
# data.columns = ['리전','월', '합계']
# data
# df_filtered.groupby('Work Type')['월'].count().reset_index()

In [45]:
# rows = df_filtered.loc[(df_filtered['월']==month)]
rows = df_filtered
    
data, pivot, total = getPivotTable_event(rows, month, region_list)
source = pivot
    
    # # Chart
    # if incompleted == 0:
    #     print('incompleted=0')
    #     chart = getPieChart(source)
    # else:
        
source = flatten_5d(data)

In [46]:
# Chart - Pie 차트
# if region == 'All':
    
#     # Data
#     rows = df_filtered.loc[(df_filtered['월']==month)]
    
#     data, pivot, total = getPivotTable_event(rows, month, region_list)
#     source = pivot
    
#     # # Chart
#     # if incompleted == 0:
#     #     print('incompleted=0')
#     #     chart = getPieChart(source)
#     # else:
        
#     source = flatten_5d(data)
    
#     chart = get_1_5_HBarChart(source)    
#     # chart.show()
#     # source = pivot
#     # if incompleted == 0:
#     #     chart = getPieChart(source)
#     # else:
#     #     chart = getStackedHBarChart1(source)
#     chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass

# Chart - 라인 차트
# data = counter_3d(df_filtered, month)
# chart = getLineChart(data)
rows = df_filtered.loc[(df_filtered['월']==month)]
    
data, pivot, total = getPivotTable_event(rows, month, region_list)
source = pivot


source = flatten_5d(data)
chart = get_1_5_HBarChart(source)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
# chart = get_1_5_StackedHBarChart(source)
chart = getLineChart2(df_filtered2, month)
# chart.show()
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [47]:
# Document
# document.add_paragraph('요청 관리', style='List Number')
document.add_heading('요청 관리', level=1) 
document.add_paragraph(f'총 {completed+incompleted}건 (완료)', style='List Bullet')
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph(f'상세 현황(워크타입별 분류)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
# if region == 'All':
#     # document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
#     document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')
# else:
#     pass

# Document
# addTable3(df_filtered, regex, month)
# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

In [48]:
document.save('demo.docx')

## 1.6 자산 관리 - VM

In [10]:
base_path = r"./data/1_6_자산관리_VM_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]
# df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_ = df_filtered[ (df_filtered['월']==month) ]
#total = len(df_)
total_instance = df_['count'].sum()

In [11]:
print(int(total_instance))

2104


In [51]:
# # Chart - Pie 차트
# if region == 'All':
    
#     # Data
#     rows = df_filtered.loc[(df_filtered['월']==month)]
    
#     data, pivot, total = getPivotTable_event(rows, month, region_list)
#     source = pivot
    
#     # Chart
#     if incompleted == 0:
#         chart = getPieChart(source)
#     else:
#         source = flatten_2d(data)
#         chart = getStackedHBarChart(source)    
    
#     # source = pivot
#     # if incompleted == 0:
#     #     chart = getPieChart(source)
#     # else:
#     #     chart = getStackedHBarChart1(source)
#     chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass

# # Chart - 라인 차트
# chart = getLineChart2s(df_filtered, month)
# chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

In [52]:
# if 'count' in df:
#     # df_filtered = df.loc[(df['월']==month)]
#     pivot = pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], values = 'count', index = ["리전", "테넌트"], columns = ["월"], aggfunc = 'sum', margins= True, margins_name='Total')
#     # pivot = pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], values = 'count', index = ["리전", "테넌트"], columns = ["월"], aggfunc = 'sum', margins= True, margins_name='Total').unstack(level=1)
# else:
#     pivot, total = None, None

In [53]:
# # pivot = pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], values = 'count', index = ["리전", "테넌트"], columns = ["월"], aggfunc = 'sum', margins= True, margins_name='Total').rename_axis('리전').reset_index()
# pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], index = ["리전", "테넌트"], aggfunc = 'sum', margins= True, margins_name='Total').rename_axis('리전').reset_index()

In [54]:
# pivot

In [55]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    
    # if 'count' in df:
    # # df_filtered = df.loc[(df['월']==month)]
    #     pivot = pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], values = 'count', index = ["리전", "테넌트"], columns = ["월"], aggfunc = 'sum', margins= True, margins_name='Total')
    # # pivot = pd.pivot_table(df_filtered[['월', '리전', '테넌트', 'count']], values = 'count', index = ["리전", "테넌트"], columns = ["월"], aggfunc = 'sum', margins= True, margins_name='Total').unstack(level=1)
    # else:
    #     pivot, total = None, None
    source = pivot
       
    # Chart
    # if incompleted == 0:
    chart = getPieChart(source)
    # else:
    #     source = flatten_2d(data)
    #     chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2s(df_filtered, month)
chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

WARN text dropped as it is incompatible with "point".


In [56]:
# chart

In [57]:
# Document
# document.add_paragraph('자산 관리(vm)', style='List Number')
document.add_heading('자산 관리(vm)', level=1) 
document.add_paragraph(f'총 {int(total_instance)}인스턴스 (운영계)', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
# addTable(df_filtered, regex, month)
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

In [58]:
document.save('demo.docx')

## 1.6 자산 관리 - DB

In [59]:
base_path = r"./data/1_6_자산관리_DB_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df_filtered, month)
pivot2, total = getPivotTable2(df_filtered, month)
pivot2 = pivot2.reset_index()

df_ = df_filtered[ (df_filtered['월']==month) ]
# chart = getStackedVBarChart1(source)
#total = len(df_)
total_instance = df_['count'].sum()

In [60]:
print(total_instance)

686.0


In [61]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

source = df_filtered
# chart = getBarChart2s(source, month)
#chart = getStackedHBarChart1(source)
chart = getStackedVBarChart1(source)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2s(source, month)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_3.png')

WARN text dropped as it is incompatible with "point".


In [62]:
df_filtered.head()

,년도,월,리전,AZ,진행상태,테넌트,유형,count,주간,월간,연간
0,22년,1월,KR,1센터,완료,PRD,PostgreSQL,110.0,O,O,O
1,22년,1월,KR,1센터,완료,PRD,Tibero,28.0,O,O,O
2,22년,1월,KR,1센터,완료,PRD,Maria,32.0,O,O,O
3,22년,1월,KR,1센터,완료,PRD,Redis,97.0,O,O,O
4,22년,1월,KR,1센터,완료,PRD,Mongo,84.0,O,O,O


In [63]:
# source = df_filtered
# bar = alt.Chart(source).mark_bar().encode(
#     x=alt.X('월:N'),
#     y=alt.Y('sum(count)', title=''),
#     color=alt.Color("유형:N", legend=None)
# )
# line = alt.Chart(source).mark_line(color="red").encode(
#     x=alt.X('월', title='월별 추세'),
#     y=alt.Y('sum(count)', title='')
# )
# text = bar.mark_text(
#     size=12,
#     align='center',
#     baseline='line-top',
#     color='black',
#     dy=-20,
#     fontSize=12,
# ).encode(
#     text = 'sum(count):Q'
# )

In [64]:
# document.add_paragraph('자산 관리(db)', style='List Number')
document.add_heading('자산 관리(db)', level=1) 
document.add_paragraph(f'총 {int(total_instance)}인스턴스(운영계)', style='List Bullet')
document.add_paragraph(f'월별 추세', style='List Bullet')


if region == 'All':
    document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
    
    # Document
    addPivot2Table(pivot2)
else:
    pass

document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
# 월별 추세 그래프 
# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

## 1.6 자산 관리 - K8s

In [41]:
base_path = r"./data/1_6_자산관리_k8s_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_month = df_filtered[ (df_filtered['월']==month) ]
total = len(df_month)

In [42]:
# df_filtered

In [43]:
df_filtered.groupby(['리전','클러스터명','프로메테우스 설치 유무'], as_index=False).count()

,리전,클러스터명,프로메테우스 설치 유무,년도,월,생성 월,AZ,진행상태,테넌트,구분,...,네트워크,담당팀,배포방식,K8S Version,API 인증서만료,CA 인증서만료,ETCD 인증서만료,주간,연간,월간
0,EU,gcseu-dkc2gpl-prd,O,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
1,EU,gcseu-svchubcore-prd,X,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
2,EU,gcseu-svchubidmz-prd,X,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
3,EU,gcseu-vtwin-prd,X,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
4,EU,local,X,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [44]:
print(total)

28


In [45]:
# Chart - Bar 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# source = df_
# chart = getBarChart(source)
# chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')
    
# Chart - Line 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_3.png')

In [68]:
# df_filtered

In [69]:
# Document
# document.add_paragraph('자산 관리(k8s)', style='List Number')
document.add_heading('자산 관리(k8s)', level=1) 
document.add_paragraph(f'총 {total}노드(운영계)', style='List Bullet')



# Chart - Bar 차트
if region == 'All':
    document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass


document.add_paragraph('월별 추세', style='List Bullet')
document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_3.png')   

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)


# Document
document.add_paragraph('상세 현황', style='List Bullet')
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTablek8s(df_filtered, regex, month)
#document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph('월별 추세', style='List Bullet')
# document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_3.png')   
document.add_page_break()

In [70]:
document.save('demo.docx')

## 1.7 용량 관리

In [71]:
base_path = r"./data/1_7_용량관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable_capacity(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

incompleted = len (df_filtered[(df_filtered['진행상태']!='완료') & (df_filtered['월']==month)])

In [72]:
print(completed, incompleted)

1 17


In [73]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('용량관리', level=1) 
    # document.add_paragraph('9. 용량관리')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 이슈: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('용량관리', level=1) 
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 이슈: {incompleted})', style='List Bullet')
    pass

# Chart - 라인 차트
# chart = getLineChart2(df_filtered, month)
# chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')
# Chart - 라인 차트
data = counter_3d(df_filtered, month)
chart = getLineChart(data)
chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')

In [74]:
data

,월,합계
0,1월,13
1,2월,8
2,3월,17
3,4월,13
4,5월,8
5,6월,12
6,7월,18
7,8월,18


In [75]:
# Document
# document.add_paragraph('용량 관리', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 이슈: {incompleted})', style='List Bullet')

# if region == 'All':
#     document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
# else:
#     pass
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   
# Document

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'자원 이슈 현황', style='List Bullet')
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable3(df_filtered, regex, month)

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   
document.add_page_break()

## 1.8 백업 관리

In [9]:
base_path = r"./data/1_8_백업관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]
# df_filtered2 = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]
df_filtered2 = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') & (df['성패']=='X')]
df_filtered3 = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') & (df['성패']=='X')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['성패']=='O') ]
completed_rows = len(df_completed)
completed = counter(df_completed, regex)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['성패']!='O') ]
incompleted_rows = len (df_incompleted)
incompleted = counter(df_incompleted, regex)

In [10]:
print(len(df_filtered3))

51


In [11]:
print(completed, incompleted)

176 51


In [12]:
# df_filtered_selectedColumns=df_filtered[['월', '리전', '유형', '진행상태', '성패', 'Instance', '비고', '월간']]
# df_filtered_selectedColumns.sort_values(by=['월', '리전', '유형'], ascending=False, inplace=True)
# df_filtered_selectedColumns.reset_index(drop=True, inplace=True)
# df_filtered=df_filtered_selectedColumns



In [13]:
# df_filtered_selectedColumns=df_filtered[['월', '리전', '유형', '진행상태', '성패', 'Instance', '비고', '월간']]
# df_filtered_selectedColumns.sort_values(by=['월', '리전', '유형'], ascending=False, inplace=True)
df_filtered.sort_values(by=['월', '리전', '유형'], ascending=False, inplace=True)
# df_filtered_selectedColumns.reset_index(drop=True, inplace=True)
df_filtered.reset_index(drop=True, inplace=True)
# df_filtered=df_filtered_selectedColumns


In [14]:
rows = df_filtered.loc[(df_filtered['월']==month)]
# rows

In [15]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart3(df_filtered, month)
#chart = getBarChart2(df_filtered, month)
chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [16]:
df_filtered2

,연도,월,리전,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간
0,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb01.ap-northeast-2.prd.h53,172.17.36.48,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
1,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb02.ap-northeast-2.prd.h53,172.17.36.107,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
2,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb03.ap-northeast-2.prd.h53,172.17.36.79,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
3,2022년,8월,KR,2센터,PRD,Mongo,완료,X,ads-ssd-mongodb01.ap-northeast-2.prd.h53,172.17.36.80,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
4,2022년,8월,KR,2센터,PRD,Mongo,완료,X,ads-ssd-mongodb02.ap-northeast-2.prd.h53,172.17.36.152,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
5,2022년,8월,KR,2센터,PRD,Mongo,완료,X,ads-ssd-mongodb03.ap-northeast-2.prd.h53,172.17.36.75,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
6,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-ssd-mongodb01.ap-northeast-2.prd.h53,172.17.36.70,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
7,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-ssd-mongodb02.ap-northeast-2.prd.h53,172.17.36.125,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
8,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-ssd-mongodb03.ap-northeast-2.prd.h53,172.17.36.205,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
9,2022년,8월,KR,2센터,PRD,Mongo,완료,X,ads-ssd-mongodb01.ap-northeast-2.prd.h53,172.17.36.80,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O


In [17]:
c=df_filtered2.groupby(['리전','비고'], as_index=False).count()
c
# df_filtered2

,리전,비고,연도,월,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,주간,월간,연간
0,KR,DB버전 낮아 백업 X,3,3,3,3,3,3,3,3,3,0,3,3,3
1,KR,담당자의 요청으로 백업 수행 안 함,3,3,3,3,3,3,3,3,3,0,3,3,3
2,KR,백업 제외 요청으로 백업 X. (백업 불필요),3,3,3,3,3,3,3,3,3,0,3,3,3
3,KR,백업 예정,3,3,3,3,3,3,3,3,3,0,3,3,3
4,KR,비표준설치(담당자설치),18,18,18,18,18,18,18,18,18,0,18,18,18
5,KR,스토리지 공간 부족으로 백업 X.,3,3,3,3,3,3,3,3,3,0,3,3,3
6,KR,제2센터는 백업 스토리지 구성 후 작업 예정,18,18,18,18,18,18,18,18,18,0,18,18,18


In [18]:
print(df_filtered2)

        연도   월  리전   AZ  테넌트          유형 진행상태 성패  \
0    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
1    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
2    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
3    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
4    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
5    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
6    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
7    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
8    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
9    2022년  8월  KR  2센터  PRD       Mongo   완료  X   
10   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
11   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
12   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
13   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
14   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
15   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
16   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
17   2022년  8월  KR  2센터  PRD       Mongo   완료  X   
126  2022년  

In [19]:
df_filtered2=df_filtered2.value_counts()
print(df_filtered2)

Series([], dtype: int64)


In [20]:
if [ month != '누적' ]:
        df_filtered2 = df_filtered2[df_filtered2['월'] == month].reset_index()
else:
    pass

    # unique_df = df['클러스터명'].value_counts()
unique_df = df_filtered2['비고'].value_counts()
    # unique_df = pd.DataFrame({'클러스터명':unique_df.index, '노드수':unique_df.values})
unique_df = pd.DataFrame({'비고':unique_df.index, 'DB 인스턴스 Qty':unique_df.values})
# unique_df = pd.DataFrame({'비고':unique_df.index, 'DB 인스턴스 Qty':unique_df.values})
df_filtered2 = df_filtered2.drop_duplicates(['비고'])

KeyError: '월'

In [18]:
unique_df

,비고,DB 인스턴스 Qty
0,비표준설치(담당자설치),31
1,제2센터는 백업 스토리지 구성 후 작업 예정,18
2,해외리전은 백업 스토리지 구성 후 작업 예정,17
3,스토리지 공간 부족으로 백업 X.,3
4,백업 예정,3
5,백업 제외 요청으로 백업 X. (백업 불필요),3
6,담당자의 요청으로 백업 수행 안 함,3
7,DB버전 낮아 백업 X,3
8,sherpa 수집서버,1


In [19]:
a=unique_df.sort_values('비고')
# a
# a['DB 인스턴스 Qty'][0]
a=a['DB 인스턴스 Qty'].reset_index()
a

,index,DB 인스턴스 Qty
0,7,3
1,8,1
2,6,3
3,5,3
4,4,3
5,0,31
6,3,3
7,1,18
8,2,17


In [20]:
# df_filtered2.sort_values('비고')
# a=unique_df.sort_values('비고')
# # a
# # a['DB 인스턴스 Qty'][0]
# a=a['DB 인스턴스 Qty'].reset_index()
df_filtered3 = df_filtered3.drop_duplicates(['비고'])
df_filtered3

,연도,월,리전,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간
0,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb01.ap-northeast-2.prd.h53,172.17.36.48,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
126,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrbigmongop01,10.11.60.231,None,스토리지 공간 부족으로 백업 X.,O,O,O
129,2022년,8월,KR,1센터,PRD,PostgreSQL,완료,X,ccskrdkc2hkpmondbp01,10.11.67.60,None,비표준설치(담당자설치),O,O,O
147,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrncontents2fcifmdbsp01,10.11.66.212,None,백업 예정,O,O,O
150,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspapigwdbcp01,10.11.57.43,None,백업 제외 요청으로 백업 X. (백업 불필요),O,O,O
154,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrsvchubmongodatap01,10.11.56.76,None,담당자의 요청으로 백업 수행 안 함,O,O,O
229,2022년,8월,EU,1센터,PRD,Mongo,완료,X,gcseusvchubmongop01,10.107.249.147,None,해외리전은 백업 스토리지 구성 후 작업 예정,O,O,O
250,2022년,8월,NA,1센터,PRD,postgreSQL,완료,X,ccsnadbmondbp01,10.112.203.225,None,sherpa 수집서버,O,O,O
304,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspdatadbp01,10.11.26.86,None,DB버전 낮아 백업 X,O,O,O


In [21]:
df_filtered3 = df_filtered3.sort_values('비고').reset_index()
# df_filtered2
# df_filtered2['DB 인스턴스 Qty']=a['DB 인스턴스 Qty']
# df_filtered3.assign(Name='DB 인스턴스 Qty')
df_filtered3

,index,연도,월,리전,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간
0,304,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspdatadbp01,10.11.26.86,None,DB버전 낮아 백업 X,O,O,O
1,250,2022년,8월,NA,1센터,PRD,postgreSQL,완료,X,ccsnadbmondbp01,10.112.203.225,None,sherpa 수집서버,O,O,O
2,154,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrsvchubmongodatap01,10.11.56.76,None,담당자의 요청으로 백업 수행 안 함,O,O,O
3,150,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspapigwdbcp01,10.11.57.43,None,백업 제외 요청으로 백업 X. (백업 불필요),O,O,O
4,147,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrncontents2fcifmdbsp01,10.11.66.212,None,백업 예정,O,O,O
5,129,2022년,8월,KR,1센터,PRD,PostgreSQL,완료,X,ccskrdkc2hkpmondbp01,10.11.67.60,None,비표준설치(담당자설치),O,O,O
6,126,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrbigmongop01,10.11.60.231,None,스토리지 공간 부족으로 백업 X.,O,O,O
7,0,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb01.ap-northeast-2.prd.h53,172.17.36.48,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O
8,229,2022년,8월,EU,1센터,PRD,Mongo,완료,X,gcseusvchubmongop01,10.107.249.147,None,해외리전은 백업 스토리지 구성 후 작업 예정,O,O,O


In [22]:
df_filtered3['DB 인스턴스 Qty']=a['DB 인스턴스 Qty']
df_filtered3

,index,연도,월,리전,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,비고,주간,월간,연간,DB 인스턴스 Qty
0,304,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspdatadbp01,10.11.26.86,None,DB버전 낮아 백업 X,O,O,O,3
1,250,2022년,8월,NA,1센터,PRD,postgreSQL,완료,X,ccsnadbmondbp01,10.112.203.225,None,sherpa 수집서버,O,O,O,1
2,154,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrsvchubmongodatap01,10.11.56.76,None,담당자의 요청으로 백업 수행 안 함,O,O,O,3
3,150,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspapigwdbcp01,10.11.57.43,None,백업 제외 요청으로 백업 X. (백업 불필요),O,O,O,3
4,147,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrncontents2fcifmdbsp01,10.11.66.212,None,백업 예정,O,O,O,3
5,129,2022년,8월,KR,1센터,PRD,PostgreSQL,완료,X,ccskrdkc2hkpmondbp01,10.11.67.60,None,비표준설치(담당자설치),O,O,O,31
6,126,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrbigmongop01,10.11.60.231,None,스토리지 공간 부족으로 백업 X.,O,O,O,3
7,0,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb01.ap-northeast-2.prd.h53,172.17.36.48,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O,18
8,229,2022년,8월,EU,1센터,PRD,Mongo,완료,X,gcseusvchubmongop01,10.107.249.147,None,해외리전은 백업 스토리지 구성 후 작업 예정,O,O,O,17


In [23]:
# df_filtered3['DB 인스턴스 Qty']=1
# df_filtered3['DB 인스턴스 Qty']=a['DB 인스턴스 Qty']
df_filtered3.rename(columns = {'비고' : '미진행 사유'}, inplace = True)
df_filtered3=df_filtered3.sort_values(['리전','AZ', '유형','미진행 사유']).reset_index(drop=True)
df_filtered3

,index,연도,월,리전,AZ,테넌트,유형,진행상태,성패,Instance,IP,백업 주기_full_archive,미진행 사유,주간,월간,연간,DB 인스턴스 Qty
0,229,2022년,8월,EU,1센터,PRD,Mongo,완료,X,gcseusvchubmongop01,10.107.249.147,None,해외리전은 백업 스토리지 구성 후 작업 예정,O,O,O,17
1,304,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspdatadbp01,10.11.26.86,None,DB버전 낮아 백업 X,O,O,O,3
2,154,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrsvchubmongodatap01,10.11.56.76,None,담당자의 요청으로 백업 수행 안 함,O,O,O,3
3,150,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrccspapigwdbcp01,10.11.57.43,None,백업 제외 요청으로 백업 X. (백업 불필요),O,O,O,3
4,147,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrncontents2fcifmdbsp01,10.11.66.212,None,백업 예정,O,O,O,3
5,126,2022년,8월,KR,1센터,PRD,Mongo,완료,X,ccskrbigmongop01,10.11.60.231,None,스토리지 공간 부족으로 백업 X.,O,O,O,3
6,129,2022년,8월,KR,1센터,PRD,PostgreSQL,완료,X,ccskrdkc2hkpmondbp01,10.11.67.60,None,비표준설치(담당자설치),O,O,O,31
7,0,2022년,8월,KR,2센터,PRD,Mongo,완료,X,aps-mongodb01.ap-northeast-2.prd.h53,172.17.36.48,None,제2센터는 백업 스토리지 구성 후 작업 예정,O,O,O,18
8,250,2022년,8월,NA,1센터,PRD,postgreSQL,완료,X,ccsnadbmondbp01,10.112.203.225,None,sherpa 수집서버,O,O,O,1


In [24]:
# df_filtered3=df_filtered3.sort_values(['리전','테넌트', '유형','비고']).reset_index()
# # df_filtered3=df_filtered3.groupby(['리전','월','테넌트','비고'])
# print(df_filtered3)

In [25]:
# Document
# document.add_paragraph('백업 관리', style='List Number')
document.add_heading('백업 관리', level=1) 

t = completed + incompleted
if t == completed :
    if t == 0 : 
        document.add_paragraph(f'총 {t} 건', style='List Bullet')
    else:
        document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
else :
    # document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')
    document.add_paragraph(f'총 {t} 건 (성공: {completed} / 실패[미진행포함]: {incompleted})', style='List Bullet')

# document.add_paragraph(f'총: {completed+incompleted}건 (성공: {completed}/실패[미진행포함]: {incompleted})', style='List Bullet')

# if region == 'All':
#    document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
# else:
#    pass

# Document
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    # addTable(df_filtered_selectedColumns)
    document.add_paragraph('상세 현황', style='List Bullet')
    # addTable_1_8(df_filtered, regex, month)
    # a=df_filtered3.sort_values('리전')
    # addTable_1_8(df_filtered2, regex, month)
    addTable_1_8(df_filtered3, regex, month)
    
    
document.add_page_break()

In [26]:
document.save('demo.docx')

## 1.9 보안 관리

In [92]:
base_path = r"./data/1_9_보안관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable_security(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [93]:
df

,년도,월,리전,AZ,테넌트,진행상태,성패,Ticket,제목,유형,주간,월간,연간
0,22년,1월,KR,1센터,DEV,완료,성공,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검,O,O,O
1,22년,1월,KR,1센터,STG,완료,성공,KRCLOUD-4134,EV 플랫폼 인프라 보안 점검 요청의 건,보안점검,O,O,O
2,22년,2월,Global,1센터,ALL,완료,성공,KRCLOUD-4377,글로벌 CCS OS 보안 설정 관련 확인 요청의 건,보안점검,O,O,O
3,22년,2월,CN,1센터,STG,완료,성공,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검,O,O,O
4,22년,2월,CN,1센터,PRD,완료,성공,CNCLOUD-1801,공신부 점검 지원 요청 드립니다.,보안점검,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,22년,7월,KR,1센터,PRD,완료,성공,KRCLOUD-5299,MongoDB 보안점검 요청 완료,보안점검,O,O,O
73,22년,8월,KR,1센터,PRD,완료,성공,KRCLOUD-5400,국내 MCP i-SAMS 점검 및 조치 요청,보안점검,O,O,O
74,22년,8월,KR,1센터,PRD,완료,성공,KRCLOUD-5396,국내 MCP i-SAMS 점검 및 조치 요청,보안점검,O,O,O
75,22년,8월,KR,1센터,PRD,완료,성공,KRCLOUD-5434,I-SAMS dbms vm 보안점검,보안점검,O,O,O


In [94]:
# Chart - Pie 차트
if region == 'All':
    
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('보안 관리', level=1) 
    # document.add_paragraph('11. 보안 관리')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('보안 관리', level=1) 
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed} / 미완료: {incompleted})', style='List Bullet')
    pass

# Chart - 라인 차트
# chart = getLineChart2(df_filtered, month)
data = counter_3d(df_filtered, month)
chart = getLineChart(data)
chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

In [95]:
# Document
# document.add_paragraph('보안 관리', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted} (완료: {completed} / 미완료: {incompleted})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

# Document
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable3(df_filtered, regex, month)

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')
document.add_page_break()

In [96]:
document.save('demo.docx')

## 1.10 하드웨어 정기점검

In [97]:
base_path = r"./data/1_10_정기점검_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [98]:
# Chart - Pie 차트
if region == 'All':
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
    document.add_heading('정기점검', level=1) 
    # document.add_paragraph('12. 정기점검')
    # Document
    document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed})', style='List Bullet')
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    document.add_heading('정기점검', level=1)
    t = completed + incompleted
    if t == completed :
        if t == 0 : 
            document.add_paragraph(f'총 {t} 건', style='List Bullet')
        else:
            document.add_paragraph(f'총 {t} 건 (완료)', style='List Bullet')
    else :
        document.add_paragraph(f'총 {t} 건 (완료: {completed} 건 / 이슈: {incompleted} 건)', style='List Bullet')

    # document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed})', style='List Bullet')
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')

In [99]:
# Document
# document.add_paragraph('정기점검', style='List Number')
# document.add_paragraph(f'총 {completed+incompleted}건 (완료: {completed})', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   

p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)


# Document
document.add_paragraph('상세 현황', style='List Bullet')
if df_filtered[df_filtered['월']==month].empty :
    pass
else:
    addTable3(df_filtered, regex, month)

# p = document.add_paragraph('')
# run = p.add_run()
# run.add_break(WD_BREAK.LINE)

# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   
document.add_page_break()

## 1.11 데이터베이스_라이선스_관리

In [100]:
base_path = r"./data/1_11_데이터베이스_라이선스_관리_v1.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') ]
#temp_license = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O') & (df['라이선스']== '임시')]
# Pivoting
pivot, total = getPivotTable(df, month)

df_ = df_filtered[ (df_filtered['월']==month) ]
#total = len(df_)

df_origin = df_filtered[(df_filtered['월']==month) & (df_filtered['라이선스']== '정식')]
origin = len(df_origin)

df_temp = df_filtered[(df_filtered['월']==month)& (df_filtered['라이선스']== '임시')]
temp = len(df_temp)
# print(total)
# df
df_completed = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']=='완료') ]
completed = len(df_completed)

df_incompleted = df_filtered[ (df_filtered['월']==month) & (df_filtered['진행상태']!='완료') ]
incompleted = len(df_incompleted)

In [101]:
df_filtered[(df_filtered['월']==month)& (df_filtered['라이선스']== '임시')]

,년도,월,리전,AZ,테넌트,진행상태,라이선스 종류,Hostname,라이선스,누적 월,비고,주간,월간,연간
1898,22년도,8월,CN,1센터,PRD,완료,MCCS,ccscnnewlmsdbp01,임시,1월,2022.10.01,O,O,O
1899,22년도,8월,CN,1센터,PRD,완료,MCCS,ccscnnewlmsdbp02,임시,1월,2022.10.01,O,O,O


In [102]:
df_filtered['라이선스 종류'] = df_filtered['라이선스 종류'].astype(str)
a=df_filtered.groupby('라이선스 종류')['월'].count().reset_index()
a.rename(columns = {'월':'합계'},inplace=True)

In [103]:
if region == 'All':
    # Data
    rows = df_filtered.loc[(df_filtered['월']==month)]
    
    data, pivot, total = getPivotTable_event(rows, month, region_list)
    source = pivot
    
    # Chart
    if incompleted == 0:
        chart = getPieChart(source)
    else:
        source = flatten_2d(data)
        chart = getStackedHBarChart(source)    
    
    
    # source = pivot

# Chart - Pie 차트
# if region == 'All':
    
#     # Data
#     rows = df_filtered.loc[(df_filtered['월']==month)]
    
#     data, pivot, total = getPivotTable_event_db_license(rows, month, region_list)
#     source = pivot
#     # chart = getPieChart(source)
    
#     # Chart
#     if incompleted == 0:
#         chart = getPieChart(source)
#     else:
#         source = flatten_d_db_license(data)
#         chart = getStackedHBarChart(source)    
    
    
#     source = pivot
#     if incompleted == 0:
#         chart = getPieChart(source)
#     else:
#         chart = getStackedHBarChart1(source)
    # chart.show()
    chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_1.png')
else:
    pass

rows = df_filtered.loc[(df_filtered['월']==month)]
    
data, pivot, total = getPivotTable_event_db_license(rows, month, region_list)
# source = pivot
source = flatten_d_db_license(data)
# chart = getStackedHBarChart(source)    
# chart = get_1_11_HBarChart(source)    


# Chart - 라인 차트
# source2 = df_filtered
# chart = getLineChart4s(source2, month)
# chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_3.png')

In [104]:
# source

In [105]:
chart = getStacked_1_11_VBarChart1(df_filtered)
# chart.show()
# source = df_filtered
# source = df_
# chart = getStackedHBarChart(source)
# chart = getBarChart4s(source, month)
# chart.show()
chart.save(f'./charts/db_license_{region}_{y}_{m}_{d}_2.png')



In [106]:
# Document
# document.add_paragraph('데이터베이스 라이선스 관리', style='List Number')
document.add_heading('데이터베이스 라이선스 관리', level=1) 
#document.add_paragraph(f'총 {total}건', style='List Bullet')
# document.add_paragraph(f'총 {origin+temp}건 (정식 라이선스: {origin}, 임시 라이선스: {temp})', style='List Bullet')
t = origin + temp
if t == origin :
    document.add_paragraph(f'총 {t} 건 (정식 라이선스)', style='List Bullet')
else :
    document.add_paragraph(f'총 {t} 건 (정식 라이선스: {origin} 건 / 임시라이선스: {temp} 건)', style='List Bullet')

if region == 'All':
    document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Document
if temp != 0 :
    document.add_paragraph('임시 라이선스 사용 내역', style='List Bullet')
    
if df_temp.empty :
    pass
else:
    addTable_DB_license(df_temp, regex, month)

# addTable(df_filtered, regex, month)
document.add_paragraph('유형별 라이선스 수', style='List Bullet')
document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_2.png')
# document.add_paragraph('월별 추세', style='List Bullet')
# document.add_picture(f'./charts/db_license_{region}_{y}_{m}_{d}_3.png')
document.add_page_break()

In [107]:
document.save('demo.docx')

## Save Document

In [108]:
document.save(f'월간보고서_{region}_contents_{y}_{m}_{d}.docx')

## 보고서 표지 (커버 쉬트)

In [109]:
# Cover Page
from docxtpl import DocxTemplate

# DRM 방지(cover-page-template.docx)
file_path=os.path.abspath("")
srcpath = findfile("cover-page-template.docx.org", file_path)
dir, file = os.path.split(srcpath)
shutil.copy2(srcpath, dir+"\cover-page-template.docx")

doc = DocxTemplate("cover-page-template.docx")
month_split = month.strip("월")

context = { 
    #'Service' : "HMC CCS Infra 운영/관리",      
    'Title' : "Monthly Report",    
    'Region' : region,        
    'Year' : "2022",
    'Month' : f"{month_split}",
    'Department' : "클라우드운영센터"
}
doc.render(context)

doc.save(f'월간보고서_cover_{region}.docx')

In [111]:
from docx import Document
from docxcompose.composer import Composer

composer = None
doc1 = Document(f'월간보고서_cover_{region}.docx')
composer = Composer(doc1)

doc2 = Document(f'월간보고서_{region}_contents_{y}_{m}_{d}.docx')
composer.append(doc2)

composer.save(f'월간보고서_{region}_{y}_{m}_{d}.docx')
# composer.save(f'{year}_{month}_MonthlyReport_{region}.docx)
